<a href="https://colab.research.google.com/github/tipiorgup/Tutorial_IDP_SAPs/blob/main/Simulation%20code/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import shutil
try:
    if os.path.exists('Tutorial_IDP_SAPs'):
        print("Found existing directory, attempting to remove...")
        shutil.rmtree('Tutorial_IDP_SAPs', ignore_errors=True)
        print("Directory removed successfully")
    else:
        print("Directory doesn't exist")
except Exception as e:
    print(f"Error while trying to remove directory: {e}")


Directory doesn't exist


In [2]:
!git clone https://github.com/tipiorgup/Tutorial_IDP_SAPs.git
!pip install openmm
!pip install MDAnalysis

Cloning into 'Tutorial_IDP_SAPs'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 115 (delta 40), reused 93 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 74.18 KiB | 10.60 MiB/s, done.
Resolving deltas: 100% (40/40), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.1 MB/s eta 0:00:00


In [3]:
os.chdir('Tutorial_IDP_SAPs/Simulation code')

In [4]:
ratio='900-100'
ionic_strength='0.3'
peptide_pair='HHHEEEWWW-HWKHWKHWK'

In [5]:
# First check file
residues_file = 'src/residues.csv'

if not os.path.exists(residues_file):
    print(f"❌ Error: File '{residues_file}' not found!")
else:
    !python clean_pair_code.py --ratio {ratio} \
        --ionic-strength {ionic_strength} \
        --ph 7.4 \
        --peptide-pair {peptide_pair} \
        --run-all \
        --residues-file 'src/residues.csv' \
        --steps 10000

Cannot import pdbfixer. This may affect 3SPN2 model.
Processing pair: HHHEEEWWW-HWKHWKHWK
Check contact with FastNS method. 
Successfully inserted 100 molecules.
Check contact with FastNS method. 
Successfully inserted 11 molecules.
/content/Tutorial_IDP_SAPs/Simulation code/src/simulation.py:98: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.038286503882254706' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  r.loc['H', 'q'] = 1. / (1 + 10**(args.ph - 6))


In [6]:
#Check in src the file parsers.py, it will show all options available